### Production Features Pipeline

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to the Feature Store at Hopsworks.ai

2. It scrapes the upcoming games for today, and saves the blank records back into the Feature Store at Hopsworks.ai so that they can be accessed by the model for the prediction service via the streamlit app.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. 

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)


DATAPATH = Path(r'data')

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 04
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=PlayIn&DateFrom=04/16/23&DateTo=04/18/23
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&DateFrom=04/16/23&DateTo=04/18/23


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Mon
Tue
[['1610612737', '1610612738'], ['1610612752', '1610612739'], ['1610612746', '1610612756']]
['42200112', '42200132', '42200172']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Access Feature Store**

In [9]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        fs = project.get_feature_store()
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [10]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg = fs.get_feature_group(
        name="rolling_stats",
        version=2,
        )
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [11]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    for j in range(tries):
        try:
            df_old = ds_query.read()
        except KeyError as e:
            if j < tries - 1: 
                time.sleep(10)
                continue
            else:
                raise ValueError('HOPSWORKS failed to connect')
        break

    if df_old.empty:
        if i < tries - 1: 
            time.sleep(10)
        else:
            raise ValueError('HOPSWORKS failed to return data')
    else:
        break



df_old


2023-04-18 07:41:15,770 INFO: USE `nba_predictor_featurestore`
2023-04-18 07:41:16,210 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_2` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2023-04-02,22201165,1610612763,1610612741,2022,107,46.406250,83.312500,44.187500,29,47,128,53.187500,86.375000,29.000000,29,36,0
1,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
2,2022-01-15,22100648,1610612746,1610612759,2021,94,38.000000,75.000000,31.593750,24,54,101,41.000000,81.500000,29.703125,19,54,0
3,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
4,2015-12-04,21500287,1610612742,1610612745,2015,96,0.457031,0.700195,0.275879,23,42,100,0.464111,0.556152,0.461914,20,45,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24008,2007-01-05,20600483,1610612763,1610612738,2006,119,0.562988,0.792969,0.300049,32,33,128,0.575195,0.691895,0.600098,25,32,0
24009,2010-04-10,20901187,1610612766,1610612765,2009,99,0.538086,0.846191,0.467041,27,35,95,0.456055,0.653809,0.285889,26,34,1
24010,2009-12-20,20900395,1610612738,1610612750,2009,122,0.519043,0.829102,0.500000,34,45,104,0.422119,0.828125,0.333008,18,41,1
24011,2010-12-10,21000334,1610612750,1610612765,2010,109,0.519043,0.817871,0.437988,21,42,99,0.385986,0.860840,0.333008,20,38,1


**Convert Feature Names back to original mixed case**

In [12]:
#hopsworks converts all feature names to lowercase, and for code reuse, we need to convert them back
df_old = convert_feature_names(df_old)
df_old
df_old[df_old['PTS_home'] == 0]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
2767,2023-04-17,42200162,1610612758,1610612744,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
3241,2023-04-17,42200122,1610612755,1610612751,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [13]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2023-04-02,22201165,1610612763,1610612741,2022,107,46.406250,83.312500,44.187500,29,47,128,53.187500,86.375000,29.000000,29,36,0
1,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
2,2022-01-15,22100648,1610612746,1610612759,2021,94,38.000000,75.000000,31.593750,24,54,101,41.000000,81.500000,29.703125,19,54,0
3,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
4,2015-12-04,21500287,1610612742,1610612745,2015,96,0.457031,0.700195,0.275879,23,42,100,0.464111,0.556152,0.461914,20,45,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24012,2023-04-17,42200162,1610612744,1610612758,2022,106,47.600000,83.300000,32.500000,18,41,114,45.700000,72.400000,23.700000,22,41,0
24013,2023-04-16,42200151,1610612747,1610612763,2022,128,53.300000,87.500000,43.200000,25,45,112,47.200000,93.800000,36.100000,24,34,1
24014,2023-04-16,42200101,1610612748,1610612749,2022,130,59.500000,71.400000,60.000000,32,36,117,49.500000,72.700000,24.400000,29,38,1
24015,2023-04-16,42200171,1610612746,1610612756,2022,115,44.100000,79.300000,32.300000,23,49,110,47.600000,78.800000,31.600000,27,42,1


**Add Today's Matchups for Feature Engineering**

In [14]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [15]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2023-04-02 00:00:00,22201165,1610612763,1610612741,2022,107.0,46.406250,83.312500,44.187500,29.0,47.0,128.0,53.187500,86.375000,29.000000,29.0,36.0,0.0,0,0.0
1,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,0,1.0
2,2022-01-15 00:00:00,22100648,1610612746,1610612759,2021,94.0,38.000000,75.000000,31.593750,24.0,54.0,101.0,41.000000,81.500000,29.703125,19.0,54.0,0.0,0,0.0
3,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,0,1.0
4,2015-12-04 00:00:00,21500287,1610612742,1610612745,2015,96.0,0.457031,0.700195,0.275879,23.0,42.0,100.0,0.464111,0.556152,0.461914,20.0,45.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24011,2023-04-17 00:00:00,42200122,1610612751,1610612755,2022,84.0,37.500000,64.700000,31.000000,24.0,33.0,96.0,45.000000,81.300000,31.400000,21.0,56.0,0.0,1,0.0
24012,2023-04-17 00:00:00,42200162,1610612744,1610612758,2022,106.0,47.600000,83.300000,32.500000,18.0,41.0,114.0,45.700000,72.400000,23.700000,22.0,41.0,0.0,1,0.0
24017,2023-04-18,42200112,1610612738,1610612737,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,0.0
24018,2023-04-18,42200132,1610612739,1610612752,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,0.0


**Feature Engineering**

In [16]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,20300012,1610612762,1610612757,2003,99,0.575195,0.713867,0.556152,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24011,2023-04-17,42200122,1610612751,1610612755,2022,84,37.500000,64.687500,31.000000,24,...,-2.571875,-2.425000,-8.000000,0.000000,1.5,-0.066667,-6.000000,-0.142857,1.0,-0.466667
24012,2023-04-17,42200162,1610612744,1610612758,2022,106,47.593750,83.312500,32.500000,18,...,5.662500,3.526042,16.000000,5.714286,4.9,3.533333,7.333333,5.000000,2.2,1.600000
24013,2023-04-18,42200112,1610612738,1610612737,2022,0,0.000000,0.000000,0.000000,0,...,7.332812,3.793750,6.333333,0.857143,1.7,0.866667,0.000000,-1.000000,-0.4,-1.800000
24014,2023-04-18,42200172,1610612756,1610612746,2022,0,0.000000,0.000000,0.000000,0,...,-5.106250,-2.519792,1.333333,1.714286,-0.3,0.733333,-5.666667,0.714286,3.7,0.933333


**Insert New Data into Feature Group**

In [17]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg.insert(df_combined, overwrite = True, write_options={"wait_for_job" : False})
    except KeyError as e:
        if i < tries - 1: 
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Uploading Dataframe: 0.00% |          | Rows 0/24016 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_2_offline_fg_backfill/executions


In [18]:
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,20300012,1610612762,1610612757,2003,99,0.575195,0.713867,0.556152,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24011,2023-04-17,42200122,1610612751,1610612755,2022,84,37.500000,64.687500,31.000000,24,...,-2.571875,-2.425000,-8.000000,0.000000,1.5,-0.066667,-6.000000,-0.142857,1.0,-0.466667
24012,2023-04-17,42200162,1610612744,1610612758,2022,106,47.593750,83.312500,32.500000,18,...,5.662500,3.526042,16.000000,5.714286,4.9,3.533333,7.333333,5.000000,2.2,1.600000
24013,2023-04-18,42200112,1610612738,1610612737,2022,0,0.000000,0.000000,0.000000,0,...,7.332812,3.793750,6.333333,0.857143,1.7,0.866667,0.000000,-1.000000,-0.4,-1.800000
24014,2023-04-18,42200172,1610612756,1610612746,2022,0,0.000000,0.000000,0.000000,0,...,-5.106250,-2.519792,1.333333,1.714286,-0.3,0.733333,-5.666667,0.714286,3.7,0.933333


In [19]:
# check to make sure there are no duplicate games were inadvertently added
df_combined[df_combined.duplicated(subset=['GAME_ID'], keep=False)]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
